RCA кратко:
происходит сужение области поиска исходя из Тор-**M** по **N** случайным точкам из области поиска.

Варианты:
- делать лучшие точки крами области поиска
- искать в пределах определенного растояния от лучших точек

In [1]:

from dipy.io.streamline import load_trk
from dipy.io.image import load_nifti
import numpy as np
from dipy.align.imaffine import (transform_centers_of_mass,
                                 AffineMap,
                                 MutualInformationMetric,
                                 AffineRegistration)
from dipy.align.transforms import (TranslationTransform3D,
                                   RigidTransform3D,
                                   AffineTransform3D)
from dipy.tracking import streamline
import copy
import pickle
import math
import simnibs
from simnibs import sim_struct, run_simnibs
from simnibs.msh import (surface,
                        mesh_io,
                        read_msh)
import random

In [2]:
# from stimvis.tms_stimulation import (my_deriv,
#                                     load_elems,
#                                     get_field,
#                                     get_ttrd,
#                                     deriv_e_field,
#                                     simulation)

def my_deriv(t,line,h=1):
    if t==0:
        return (-3*line[t]+4*line[t+1]-line[t+2])/(2*h)
    elif t==len(line)-1:
        return (line[t-2]-4*line[t-1]+3*line[t])/(2*h)
    else:
        return (line[t+1]-line[t-1])/(2*h)

def load_elems(nodes,elems):

    import numpy as np

    elems = elems[elems[:,3]!= -1,:]
    # Computing rectangles
    tmp = nodes[elems-1,:]
    elems_min = tmp.min(axis=1)
    elems_max = tmp.max(axis=1)
    tmp = 0
    sizes = (elems_max-elems_min).max(axis=0)
    # It is the index to reduce the elements to check
    order_min = np.argsort(elems_min[:,0])
    return {"Nodes":nodes, "Elems":elems, "El_min":elems_min, "El_max":elems_max, "Sizes":sizes,"Order_min":order_min}

# a function to get e-field vector at a given position [x,y,z]
def get_field(ttt, point, my_field):
    best_ttt=get_ttrd(ttt,point)
    return np.dot(my_field[best_ttt['Nodes']-1].T, best_ttt['Weights'])

def get_ttrd(loaded_elems,point):
    import numpy as np
    # Just to use names I have used before
    nodes = loaded_elems["Nodes"]
    elems = loaded_elems["Elems"]
    elems_min = loaded_elems["El_min"]
    elems_max = loaded_elems["El_max"]
    sizes = loaded_elems["Sizes"]
    order_min = loaded_elems["Order_min"]
    
    # Binary search to reduce the iterating points from 4mln to around 200k.
    r = np.searchsorted(elems_min[:,0],point[0],side='left',sorter=order_min)
    l = np.searchsorted(elems_min[:,0],point[0] - sizes[0],side='right',sorter=order_min)
    # Projection the data to only these points
    e_max = elems_max[order_min[l:r]]
    e_min = elems_min[order_min[l:r]]
    
    # Checks which ttrds are possible to contain the point
    potential_ttrds = order_min[l:r][(point[0] <= e_max[:,0]) & (e_min[:,1]<= point[1]) & (point[1] <= e_max[:,1]) & (e_min[:,2]<= point[2]) & (point[2] <= e_max[:,2])]
    
    # It checks if the ttrd contains the point
    def check_ttrd(ttrd, point):
        coord = np.column_stack((ttrd[1,:]-ttrd[0,:],ttrd[2,:]-ttrd[0,:],ttrd[3,:]-ttrd[0,:]))
        coord = np.linalg.inv(coord).dot(point-ttrd[0,:])
        return coord.min() >= 0 and coord.sum() <= 1
    # It checks if the ttrd with num ttrdNum contains the point
    def check_ttrd_byNum(ttrdNum, point):
        ttrd = nodes[elems[ttrdNum]-1]
        return check_ttrd(ttrd,point)
    
    # Just takes all ttrds that contain points
    nodeIndices = elems[[x for x in potential_ttrds if check_ttrd_byNum(x,point)]][0]; 
    ns = nodes[nodeIndices-1]

    norms = np.sum((ns-point)**2,axis=-1)**0.5
    weights = 1/(norms+1e-10)
    weights = weights / weights.sum()
    
    return {"Nodes":nodeIndices,"Weights":weights}


# a function to calculate directional derivatives of the effective field at a given point [x,y,z]
def deriv_e_field(coordinates, e_field_nodes, LSD, ttt):
   
    step=0.05

    x1=coordinates[0]
    y1=coordinates[1]
    z1=coordinates[2]
    x0=coordinates[0]-step
    x2=coordinates[0]+step
    y0=coordinates[1]-step
    y2=coordinates[1]+step
    z0=coordinates[2]-step
    z2=coordinates[2]+step

    f_x0_y1_z1=np.dot(get_field(ttt,np.asarray([x0,y1,z1]), e_field_nodes), LSD)
    f_x2_y1_z1=np.dot(get_field(ttt,np.asarray([x2,y1,z1]), e_field_nodes), LSD)
    f_x1_y1_z1=np.dot(get_field(ttt,np.asarray([x1,y1,z1]), e_field_nodes), LSD)
    f_x1_y0_z1=np.dot(get_field(ttt,np.asarray([x1,y0,z1]), e_field_nodes), LSD)
    f_x1_y2_z1=np.dot(get_field(ttt,np.asarray([x1,y2,z1]), e_field_nodes), LSD)
    f_x1_y1_z0=np.dot(get_field(ttt,np.asarray([x1,y1,z0]), e_field_nodes), LSD)
    f_x1_y1_z2=np.dot(get_field(ttt,np.asarray([x1,y1,z2]), e_field_nodes), LSD)
    
    gradx=my_deriv(1,[f_x0_y1_z1,f_x1_y1_z1,f_x2_y1_z1], step)
    grady=my_deriv(1,[f_x1_y0_z1,f_x1_y1_z1,f_x1_y2_z1], step)
    gradz=my_deriv(1,[f_x1_y1_z0,f_x1_y1_z1,f_x1_y1_z2], step)
    
    return np.dot([gradx, grady, gradz], LSD)

In [23]:
def simulation(fnamehead, pathfem, pos_centre=[-74.296158, -10.213354, 28.307243], pos_ydir=[-74.217369, -37.293205, 20.05232], distance=4, current_change=1e6):
    # Initalize a session
    s = sim_struct.SESSION()
    # Name of head mesh
    s.fnamehead = fnamehead
    # Output folder
    s.pathfem = pathfem
    # Not to visualize results in gmsh when running simulations (else set to True)
    s.open_in_gmsh=False
    
    # Initialize a list of TMS simulations
    tmslist = s.add_tmslist()
    # Select coil. For full list of available coils, please see simnibs documentation
    tmslist.fnamecoil = 'Magstim_70mm_Fig8.nii.gz'
    
    # Initialize a coil position
    pos = tmslist.add_position()
    pos.centre = pos_centre # Place the coil over
    pos.pos_ydir = pos_ydir # Point the coil towards
    pos.distance = distance # Distance between coil and head
    pos.didt = current_change # Rate of change of current in the coil, in A/s.
    run_simnibs(s)

def calc_TMS_effects(x, y, z, pos_ydir): # (change_TMS_effects)
    
    l1 = 2  # membrane space constant 2mm
    l2 = l1**2
    
    # x,y,z already projected on scalp and in right coordinate system.
    # that's why next line commented
    #position = [x-256/2, y-256/2, z-256/2]  # -256/2 because of a freesurfer RAS coordinate system
    current_out_dir = out_dir+str(x)+'_'+str(y)+'_'+str(z)
    try:
        simulation(mesh_path, current_out_dir, pos_centre=[x,y,z], pos_ydir=pos_ydir)
    except Exception as ex:
        print('EROR: simulation(..) in calc_TMS_effects \n', ex)

    mesh_file = current_out_dir+'/'+subject_name+'_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh'
    field_mesh = read_msh(mesh_file)
    field_as_nodedata = field_mesh.elmdata[0].as_nodedata()
    field_at_nodes = field_as_nodedata.value
    ttt = load_elems(field_mesh.nodes.node_coord, field_mesh.elm.node_number_list)

    effective_field = copy.deepcopy(new_streams_T1_array)

    for stream in range(len(new_streams_T1_array)):
        my_steam = copy.deepcopy(new_streams_T1_array[stream])
        print('starting _'+str(stream)+' out of '+str(len(new_streams_T1_array)))
        for t in range(len(my_steam[:, 0])):
            # -256/2 because of a freesurfer RAS coordinate system
            x = my_steam[t, 0]-256/2
            y = my_steam[t, 1]-256/2
            z = my_steam[t, 2]-256/2
            xyz = np.asarray([x, y, z])

            field_vector_xyz = get_field(ttt, xyz, field_at_nodes)

            effective_field[stream][t, 0] = l1*np.dot(field_vector_xyz, streams_array_derivative[stream][t, :])
            effective_field[stream][t, 1] = l2*deriv_e_field(xyz, field_at_nodes, streams_array_derivative[stream][t, :], ttt)
            effective_field[stream][t, 2] = effective_field[stream][t, 0] + effective_field[stream][t, 1]
            
    with open(current_out_dir+'/'+subject_name+'_effective_field.txt', 'wb') as f:
        pickle.dump(effective_field, f)
    f.close()
    # Поиск максимума среди всех точек:
    return max([max(effective_field[stream][:,2]) for stream in range(len(effective_field))])

In [4]:
# Ввод данных
tractography_file='Data/tracts.trk'

sft = load_trk(tractography_file, tractography_file)
streams = sft.streamlines
streams_array = np.asarray(streams)
print('imported tractography data:'+tractography_file)

fname_T1 = 'Data/T1fs_conform.nii.gz'#input("Please, specify the T1fs_conform image that has been generated during head meshing procedure. ")
data_T1, affine_T1 = load_nifti(fname_T1)

fname_FA = 'Data/dti_fa.nii' #input("Please, specify the FA image. ")
data_FA, affine_FA = load_nifti(fname_FA)
print('loaded T1fs_conform.nii and FA images')

mesh_path = 'Data/subject.msh' #input("Please, specify the head mesh file. ")

last_slach = max([i for i, ltr in enumerate(mesh_path) if ltr == '/'])+1
subject_name = mesh_path[last_slach:-4]


# specify the directory where you would like to save your simulation results
# input example:'/home/Example_data/Output'
out_dir = 'Output' #input("Please, specify the directory where you would like to save your simulation results. ")
out_dir = out_dir+'/simulation_at_pos_'

imported tractography data:Data/tracts.trk
loaded T1fs_conform.nii and FA images


In [5]:
%%time
# Co-registration of T1fs_conform and FA images. Performed in 4 steps.
# Step 1. Calculation of the center of mass transform. Used later as starting transform.
c_of_mass = transform_centers_of_mass(data_T1, affine_T1, data_FA, affine_FA)
print('calculated c_of_mass transformation')

# Step 2. Calculation of a 3D translation transform. Used in the next step as starting transform.
nbins = 32
sampling_prop = None
metric = MutualInformationMetric(nbins, sampling_prop)
level_iters = [10000, 1000, 100]
sigmas = [3.0, 1.0, 0.0]
factors = [4, 2, 1]
affreg = AffineRegistration(metric=metric,
                            level_iters=level_iters,
                            sigmas=sigmas,
                            factors=factors)

transform = TranslationTransform3D()
params0 = None
starting_affine = c_of_mass.affine
translation = affreg.optimize(data_T1, data_FA, transform, params0,
                    affine_T1, affine_FA,
                    starting_affine=starting_affine)
print('calculated 3D translation transform')

# Step 3. Calculation of a Rigid 3D transform. Used in the next step as starting transform
transform = RigidTransform3D()
params0 = None
starting_affine = translation.affine
rigid = affreg.optimize(data_T1, data_FA, transform, params0, 
                        affine_T1, affine_FA,
                        starting_affine=starting_affine)
print('calculated Rigid 3D transform')

# Step 4. Calculation of an affine transform. Used for co-registration of T1 and FA images.
transform = AffineTransform3D()
params0 = None
starting_affine = rigid.affine
affine = affreg.optimize(data_T1, data_FA, transform, params0,
                         affine_T1, affine_FA,
                         starting_affine=starting_affine)

print('calculated Affine 3D transform')

identity = np.eye(4)

inv_affine_FA = np.linalg.inv(affine_FA)
inv_affine_T1 = np.linalg.inv(affine_T1)
inv_affine = np.linalg.inv(affine.affine)

# transforming streamlines to FA space
new_streams_FA = streamline.transform_streamlines(streams, inv_affine_FA)
new_streams_FA_array = np.asarray(new_streams_FA)

T1_to_FA = np.dot(inv_affine_FA, np.dot(affine.affine, affine_T1))
FA_to_T1 = np.linalg.inv(T1_to_FA)

# transforming streamlines from FA to T1 space
new_streams_T1 = streamline.transform_streamlines(new_streams_FA, FA_to_T1)
new_streams_T1_array = np.asarray(new_streams_T1)

# calculating amline derivatives along the streamlines to get the local orientation of the streamlines
streams_array_derivative = copy.deepcopy(new_streams_T1_array)

print('calculating streamline derivatives')
for stream in range(len(new_streams_T1_array)):
    my_steam = new_streams_T1_array[stream]
    for t in range(len(my_steam[:, 0])):
        streams_array_derivative[stream][t, 0] = my_deriv(t, my_steam[:, 0])
        streams_array_derivative[stream][t, 1] = my_deriv(t, my_steam[:, 1])
        streams_array_derivative[stream][t, 2] = my_deriv(t, my_steam[:, 2])
        deriv_norm = np.linalg.norm(streams_array_derivative[stream][t, :])
        streams_array_derivative[stream][t, :] = streams_array_derivative[stream][t, :]/deriv_norm    
        
#------ Чтоб не повторять по нескольку раз и немного экономить время (потом удалить)
with open('../new_streams_T1_array.txt', 'wb') as f:
    pickle.dump(new_streams_T1_array, f)
f.close()
with open('../streams_array_derivative.txt', 'wb') as f:
    pickle.dump(streams_array_derivative, f)
f.close()


calculated c_of_mass transformation
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
calculated 3D translation transform
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
calculated Rigid 3D transform
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
calculated Affine 3D transform
calculating streamline derivatives
CPU times: user 20min 25s, sys: 37.4 s, total: 21min 2s
Wall time: 16min 19s


In [5]:
with open('../new_streams_T1_array.txt', 'rb') as f:
    new_streams_T1_array = pickle.load(f)
f.close()

with open('../streams_array_derivative.txt', 'rb') as f:
    streams_array_derivative = pickle.load(f)
f.close()

In [6]:
mesh_struct = mesh_io.read_msh(mesh_path)
scalp=surface.Surface(mesh_struct, [5,1005])

def get_coords_from_angles(alfa, beta):
    # (1) Преобразование в трехмерную систему координат. x,y,z ∈ [0,255]
    r=256/2
    a= math.radians(alfa)
    b= math.radians(beta)
    
    x= r * math.sin(a) * math.cos(b) +r
    y= r * math.cos(a) +r
    z= r * math.sin(a) * math.sin(b) +r
    
    coords=np.array((x,y,z))
    coords-=256/2
    
    # (2) Получение проекции координаты на голову
    proj_coords,_=scalp.projectPoint(coords)
    return proj_coords

----
## Основная часть

По системам координат:
 1) углы сферической --> x,y,z

In [7]:
import multiprocessing
from os.path import exists

In [8]:
class Point:
    def __init__(self, angles):
        self.angles=angles
        self.coords=get_coords_from_angles(angles[0], angles[1]).tolist()
        
        
    def __lt__(self, other):
        return self.fitness > other.fitness
    
    def calc_fitness(self):
        try:
            x,y,z=self.coords
            path_to_possible_effective_field=out_dir+str(x)+'_'+str(y)+'_'+str(z)+'/'+subject_name+'_effective_field.txt'
            
            # if simulation on this coordinates somehow was done earlier
            if exists(path_to_possible_effective_field):
                with open(path_to_possible_effective_field, 'rb') as f:
                    effective_field = pickle.load(f)
                f.close()
                self.fitness=max([max(effective_field[stream][:,2]) for stream in range(len(effective_field))])
            
            # normal way
            else:
                rotation_angle=math.radians(self.angles[2])
                pos_ydir=get_coords_from_angles(self.angles[0] + math.sin(rotation_angle)*7, # just 7
                                                self.angles[1] + math.cos(rotation_angle)*7)
                #print('pos_ydir =', pos_ydir)
                self.fitness=calc_TMS_effects(*self.coords, pos_ydir)
        except Exception as ex:
            print('EROR_EROR_SO_SAD_EROR', self.coords)
            print(ex)
    
    def is_point_close_to_streams(self):
        return (self.coords[0]>=x_min and self.coords[0]<=x_max 
                and self.coords[1]>=y_min and self.coords[1]<=y_max 
                and self.coords[2]>=z_min and self.coords[2]<=z_max)

In [9]:
class Consumer(multiprocessing.Process):

    def __init__(self, task_queue, result_queue):
        multiprocessing.Process.__init__(self)
        self.task_queue = task_queue
        self.result_queue = result_queue

    def run(self):
        pname = self.name
            
        while True:
            point = self.task_queue.get()
            
            if point is None: # poison pill
                print('Exiting %s...' % pname)
                self.task_queue.task_done()
                break
                
            print('{0} processing task: {1}'.format(pname, point.coords))
            try:
                point.calc_fitness()
            except Exception as ex:
                print('=================ERRRROOOORRRRR RRRRRR!!!========= eror')
            
            self.task_queue.task_done()
            self.result_queue.put(point)

In [10]:
# Ограничение в 25мм
x_min = np.min(new_streams_T1_array[0][:,0]) - 25 - 256/2
x_max = np.max(new_streams_T1_array[0][:,0]) + 25 - 256/2
y_min = np.min(new_streams_T1_array[0][:,1]) - 25 - 256/2
y_max = np.max(new_streams_T1_array[0][:,1]) + 25 - 256/2
z_min = np.min(new_streams_T1_array[0][:,2]) - 25 - 256/2
z_max = np.max(new_streams_T1_array[0][:,2]) + 25 - 256/2
print (x_min, x_max, y_min, y_max, z_min, z_max)

-58.35510653248957 6.999714416924746 -37.13407266408484 29.310656427633916 -31.340430584812097 93.1118822930465


----

In [15]:
# Settings:
search_area={'alfa':[0,360], 'beta':[0,180], 'rotation':[0,360]}
number_of_trials = 20
last_steps_start_with = 14
number_in_trial = 10
n_best = 5
best_point=None

In [49]:
# Можно удалить.
from ipywidgets import IntProgress
from IPython.display import display
prg_bar = IntProgress(min = 0, max = number_of_trials) # Создаем прогрессбар
display(prg_bar)

IntProgress(value=0, max=10)

In [56]:
%%time
# RCA with mp Last version
#search_area={'alfa':[0,180], 'beta':[0,180], 'rotation':[0,360]}

number_of_trials = 10
last_steps_start_with = 7
number_in_trial = 10
n_best = 5
#best_point=None
#step_points=[]

for current_trial in range(prg_bar.value, number_of_trials):
    # Generate random angles from search_area
    points.clear()
    if current_trial < last_steps_start_with:
        while len(points)<number_in_trial:
            point=Point([random.uniform(*search_area['alfa']),
                         random.uniform(*search_area['beta']),
                         random.uniform(*search_area['rotation'])])
            if point.is_point_close_to_streams():
                points.append(point)

    else:
        while len(points)<int(number_in_trial*0.8):
            point=Point([random.uniform(*search_area['alfa']),
                         random.uniform(*search_area['beta']),
                         random.uniform(*search_area['rotation'])])
            if point.is_point_close_to_streams():
                points.append(point)
        
        while len(points)<number_in_trial:
            point=Point([random.uniform(0, 180),
                         random.uniform(0, 180),
                         random.uniform(0, 360)])
            if point.is_point_close_to_streams():
                points.append(point)
    
    # multiprocessing calculation
    if __name__ == '__main__':
        tasks = multiprocessing.JoinableQueue()
        results = multiprocessing.Queue()
        # spawning consumers
        n_consumers = 2 #multiprocessing.cpu_count()
        print('Spawning %i consumers...' % n_consumers)
        consumers = [Consumer(tasks, results) for i in range(n_consumers)]

        for consumer in consumers:
            consumer.start()

        # enqueueing jobs
        for point in points:
            tasks.put(point)
        # poison pill
        for i in range(n_consumers):
            tasks.put(None)
        
        tasks.join()
        
        step_points=[results.get() for i in range(len(angles_list))]
    
    step_points.sort()
    if best_point is None or best_point.fitness<step_points[0].fitness:
        best_point=step_points[0]
    # Override search area
    # sort points by effect
    for point in step_points[:n_best]:
        print(point.fitness, point.coords)
    search_area['alfa'] = [min(step_points[:n_best], key=lambda point : point.angles[0]).angles[0], #key=alfa
                           max(step_points[:n_best], key=lambda point : point.angles[0]).angles[0]]
    search_area['beta'] = [min(step_points[:n_best], key=lambda point : point.angles[1]).angles[1], #key=beta
                           max(step_points[:n_best], key=lambda point : point.angles[1]).angles[1]]
    search_area['rotation'] = [min(step_points[:n_best], key=lambda point : point.angles[2]).angles[2], #key=rotation
                               max(step_points[:n_best], key=lambda point : point.angles[2]).angles[2]]
    
    prg_bar.value+=1 # !prg_bar! delme
    print('Search area:', search_area)
print('RESULT:', best_point.coords, best_point.fitness)

Spawning 2 consumers...
Consumer-33 processing task: [-19.831064, -15.802687, 84.17701]
Consumer-34 processing task: [-19.831064, -15.802687, 84.17701]


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_-19.831064_-15.802687_84.17701
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_-19.831064_-15.802687_84.17701
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_-

[ simnibs ]INFO: Result Files:
/home/roman/Документы/Notebooks/Output/simulation_at_pos_-19.831064_-15.802687_84.17701/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
INFO:simnibs:Result Files:
/home/roman/Документы/Notebooks/Output/simulation_at_pos_-19.831064_-15.802687_84.17701/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
[ simnibs ]INFO: =====================================
INFO:simnibs:=====================================
[ simnibs ]INFO: SimNIBS finished running simulations
INFO:simnibs:SimNIBS finished running simulations
[ simnibs ]INFO: Simulation Result Meshes:
INFO:simnibs:Simulation Result Meshes:
[ simnibs ]INFO: /home/roman/Документы/Notebooks/Output/simulation_at_pos_-19.831064_-15.802687_84.17701/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
INFO:simnibs:/home/roman/Документы/Notebooks/Output/simulation_at_pos_-19.831064_-15.802687_84.17701/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
[ simnibs ]INFO: =====================================
INFO:

starting _0 out of 6
starting _0 out of 6
starting _1 out of 6
starting _1 out of 6
starting _2 out of 6
starting _2 out of 6
starting _3 out of 6
starting _3 out of 6
starting _4 out of 6
starting _4 out of 6
starting _5 out of 6
starting _5 out of 6
Consumer-34 processing task: [-15.737746, -14.846397, 85.423958]


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None


...

starting _0 out of 6
starting _0 out of 6
starting _1 out of 6
starting _1 out of 6
starting _2 out of 6
starting _2 out of 6
starting _3 out of 6
starting _3 out of 6
starting _4 out of 6
starting _4 out of 6
starting _5 out of 6
starting _5 out of 6
Consumer-33 processing task: [-50.447338, -33.618401, 64.442574]
Consumer-34 processing task: [4.516225, 22.374262, 85.379448]


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_-50.447338_-33.618401_64.442574
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_-50.447338_-33.618401_64.442574
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos

INFO:simnibs:Result Files:
/home/roman/Документы/Notebooks/Output/simulation_at_pos_-50.447338_-33.618401_64.442574/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
[ simnibs ]INFO: =====================================
INFO:simnibs:=====================================
[ simnibs ]INFO: SimNIBS finished running simulations
INFO:simnibs:SimNIBS finished running simulations
[ simnibs ]INFO: Simulation Result Meshes:
INFO:simnibs:Simulation Result Meshes:
[ simnibs ]INFO: /home/roman/Документы/Notebooks/Output/simulation_at_pos_-50.447338_-33.618401_64.442574/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
INFO:simnibs:/home/roman/Документы/Notebooks/Output/simulation_at_pos_-50.447338_-33.618401_64.442574/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
[ simnibs ]INFO: =====================================
INFO:simnibs:=====================================
[ simnibs ]SUMMARY: 
subject_TMS_1-0001_Magstim_70mm_Fig8_nii
Gray Matter

Field Percentiles
-----------------
Top percent

starting _0 out of 6
starting _0 out of 6
starting _1 out of 6
starting _1 out of 6
starting _2 out of 6
starting _2 out of 6
starting _3 out of 6
starting _3 out of 6
starting _4 out of 6
starting _4 out of 6
starting _5 out of 6
starting _5 out of 6
Exiting Consumer-34...
Exiting Consumer-33...


AttributeError: 'Point' object has no attribute 'fitness'

In [ ]:
# Предыдущий блок упал по неизвестной причине. Поэтому он был перезапущен с сохранением области поиска и шага.
print(step_points[0].fitness)

In [59]:
best_point.fitness

5.280954818545432

In [58]:
len(step_points)

10

In [57]:
for x in step_points:
    print(x.fitness, x.coords)

4.76289338241086 [-19.831064, -15.802687, 84.17701]
4.76289338241086 [-19.831064, -15.802687, 84.17701]
4.511410332957293 [-16.342142, -17.870005, 85.382187]
4.335636547998097 [-15.737746, -14.846397, 85.423958]


AttributeError: 'Point' object has no attribute 'fitness'

In [45]:
print(step_points[0].fitness)


5.280954818545432


3.4778625275136648

In [41]:
# запуск проблемного элемента для выяснения ошибки
xx= copy.deepcopy(step_points[7])
xx.calc_fitness()

[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_-16.664179_-20.530813_85.310478
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_-16.664179_-20.530813_85.310478
[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_-16.664179_-20.530813_85.310478
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_-16.664179_-20.530813_85.310478
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnib

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6


In [42]:
xx.fitness

4.7238498382862

In [158]:
# single process 
#prg_bar.value=0
points=[]
for current_trial in range(number_of_trials):
    points.clear()
    for current_point in range(number_in_trial):
        if current_trial < last_steps_start_with or current_point<number_in_trial*0.8:
            points.append(point(random.uniform(*search_area['alfa']),
                                random.uniform(*search_area['beta'])))
        else:
            points.append(point(random.uniform(0,180),
                                random.uniform(0,180)))
        #prg_bar.value+=1
    points.sort()
    # Переопределить область поиска
    search_area['alfa'] = [min(points[:n_best], key=lambda x : x.alfa).alfa,
                           max(points[:n_best], key=lambda x : x.alfa).alfa]
    search_area['beta'] = [min(points[:n_best], key=lambda x : x.beta).beta,
                           max(points[:n_best], key=lambda x : x.beta).beta]
    print(search_area)
    # search_area['rotation']=...
print('RESULT:',points[0].coords)

{'alfa': [33.15147903779016, 86.17575286442482], 'beta': [123.28673063270821, 138.4796600042052], 'rotation': [0, 360]}
{'alfa': [50.56189333081386, 85.89564505068928], 'beta': [123.32307723822487, 137.0251573622522], 'rotation': [0, 360]}
{'alfa': [50.989523113708394, 81.52199687992285], 'beta': [126.76145661826412, 134.4126521789578], 'rotation': [0, 360]}
{'alfa': [71.67219983858294, 81.11385901273715], 'beta': [128.71250207876042, 132.58689391160715], 'rotation': [0, 360]}
{'alfa': [73.6387627070121, 80.82278064361437], 'beta': [129.23169406459513, 132.4602068754683], 'rotation': [0, 360]}
{'alfa': [73.93184921291912, 80.15056163688745], 'beta': [129.3610610071763, 132.1418958571237], 'rotation': [0, 360]}
{'alfa': [76.62168664587325, 80.04607353387519], 'beta': [129.5847078523424, 130.88698451317828], 'rotation': [0, 360]}
{'alfa': [77.33750825667299, 79.14402324848116], 'beta': [129.59274215334517, 130.48839488022008], 'rotation': [0, 360]}
{'alfa': [77.55974921850752, 78.7964978

### Тест на скорость выполнения 10 симуляций с разным количеством задействованных ядер
- Without multiprocessing: 22min 55s
- 1 core: 23min 10s
- 2 cores: 15min 13s
- 3 cores: 19min 4s, 18min, 59s
- 4 cores: browser dropped

In [12]:
%%time
# Without multiprocessing
result=[]

my_input = [(random.uniform(0,100),random.uniform(0,100)) for x in range(10)]

result=[calc_data_for_point(x) for x in my_input]

[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_72.418297_37.32938_36.187672
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_72.418297_37.32938_36.187672
[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_72.418297_37.32938_36.187672
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_72.418297_37.32938_36.187672
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began to r

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_31.251886_48.24506_68.964783
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_31.251886_48.24506_68.964783
[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_31.251886_48.24506_68.964783
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_31.251886_48.24506_68.964783
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began to r

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_69.980797_42.794258_35.591167
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_69.980797_42.794258_35.591167
[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_69.980797_42.794258_35.591167
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_69.980797_42.794258_35.591167
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began 

...

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_52.979767_58.636154_41.787472
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_52.979767_58.636154_41.787472
[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_52.979767_58.636154_41.787472
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_52.979767_58.636154_41.787472
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began 

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_43.767292_74.524979_27.608877
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_43.767292_74.524979_27.608877
[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_43.767292_74.524979_27.608877
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_43.767292_74.524979_27.608877
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began 

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
CPU times: user 38min 28s, sys: 2min 22s, total: 40min 51s
Wall time: 22min 55s


In [13]:
%%time
# An unsuccessful attempt to perform a parallel calculation
# Everything was done with a single core
# 23min 10s
result=[]

if __name__ == '__main__':
    manager = multiprocessing.Manager()
    results = manager.Queue()
    tasks = multiprocessing.JoinableQueue()
    #results = multiprocessing.Queue()

    # spawning consumers with respect to the
    # number cores available in the system
    n_consumers = multiprocessing.cpu_count()
    print('Spawning %i consumers...' % n_consumers)
    consumers = [Consumer(tasks, results) for i in range(n_consumers)]
    for consumer in consumers:
        consumer.start()

    # enqueueing jobs
    my_input = [(random.uniform(0,100),random.uniform(0,100)) for x in range(10)]
    for item in my_input:
        tasks.put(item)
        
    tasks.join()
    print('-----')
    result =[results.get() for i in range(len(my_input))]

    print('Done.')
print('Code, please be faster')

Spawning 4 consumers...
Consumer-10 processing task:


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_38.650665_50.47953_63.041931
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_38.650665_50.47953_63.041931
[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_38.650665_50.47953_63.041931
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_38.650665_50.47953_63.041931
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began to r

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
Consumer-10 processing task:


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_35.138245_56.388012_60.656631
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_35.138245_56.388012_60.656631
[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_35.138245_56.388012_60.656631
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_35.138245_56.388012_60.656631
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began 

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
Consumer-10 processing task:


...

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
Consumer-10 processing task:


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_42.123013_49.144829_61.166168
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_42.123013_49.144829_61.166168
[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_42.123013_49.144829_61.166168
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_42.123013_49.144829_61.166168
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began 

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
Consumer-10 processing task:


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_63.862919_48.604305_38.78373
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_63.862919_48.604305_38.78373
[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_63.862919_48.604305_38.78373
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_63.862919_48.604305_38.78373
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began to r

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
-----
Done.
Code, please be faster
CPU times: user 305 ms, sys: 181 ms, total: 485 ms
Wall time: 23min 10s


Using a poison pill

In [42]:
%%time
# mulriprocessing 2 cores
result=[]

if __name__ == '__main__':
    #manager = multiprocessing.Manager()
    #results = manager.Queue()
    tasks = multiprocessing.JoinableQueue()
    results = multiprocessing.Queue()

    # spawning consumers with respect to the
    # number cores available in the system -1
    n_consumers = multiprocessing.cpu_count()-2
    print('Spawning %i consumers...' % n_consumers)
    consumers = [Consumer(tasks, results) for i in range(n_consumers)]
    for consumer in consumers:
        consumer.start()

    # enqueueing jobs
    my_input = [(random.uniform(0,100),random.uniform(0,100)) for x in range(10)]
    for item in my_input:
        tasks.put(item)
    # poison pill
    for i in range(n_consumers):
        tasks.put(None)
    
    tasks.join()
    
    result =[results.get() for i in range(len(my_input))]
    print('Done.')

Spawning 2 consumers...
Consumer-67 processing task:
Consumer-68 processing task:


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_40.139816_76.252769_27.867176
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_40.139816_76.252769_27.867176
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_68.

INFO:simnibs:Result Files:
/home/roman/Документы/Notebooks/Output/simulation_at_pos_40.139816_76.252769_27.867176/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
[ simnibs ]INFO: =====================================
INFO:simnibs:=====================================
[ simnibs ]INFO: SimNIBS finished running simulations
INFO:simnibs:SimNIBS finished running simulations
[ simnibs ]INFO: Simulation Result Meshes:
INFO:simnibs:Simulation Result Meshes:
[ simnibs ]INFO: /home/roman/Документы/Notebooks/Output/simulation_at_pos_40.139816_76.252769_27.867176/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
INFO:simnibs:/home/roman/Документы/Notebooks/Output/simulation_at_pos_40.139816_76.252769_27.867176/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
[ simnibs ]INFO: =====================================
INFO:simnibs:=====================================
[ simnibs ]SUMMARY: 
subject_TMS_1-0001_Magstim_70mm_Fig8_nii
Gray Matter

Field Percentiles
-----------------
Top percentiles o

starting _0 out of 6
starting _0 out of 6
starting _1 out of 6
starting _1 out of 6
starting _2 out of 6
starting _2 out of 6
starting _3 out of 6
starting _3 out of 6
starting _4 out of 6
starting _4 out of 6
starting _5 out of 6
starting _5 out of 6
Consumer-68 processing task:


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_70.763947_41.499119_35.410679
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_70.763947_41.499119_35.410679


...

[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_69.260773_32.842316_43.797318
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_69.260773_32.842316_43.797318
[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_69.260773_32.842316_43.797318
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_69.260773_32.842316_43.797318
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began 

Consumer-68 processing task:


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_59.426521_52.856895_40.114452
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_59.426521_52.856895_40.114452
[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_59.426521_52.856895_40.114452
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_59.426521_52.856895_40.114452
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began 

SUMMARY:simnibs:
subject_TMS_1-0001_Magstim_70mm_Fig8_nii
Gray Matter

Field Percentiles
-----------------
Top percentiles of the field (or field norm for vector fields)
|Field |99.9%        |99.0%        |95.0%        |
|------|-------------|-------------|-------------|
|E     |7.08e-01 V/m |4.97e-01 V/m |2.95e-01 V/m |
|normE |7.08e-01 V/m |4.97e-01 V/m |2.95e-01 V/m |

Field Focality
---------------
Mesh volume or area with a field >= X% of the 99.9th percentile
|Field |75.0%        |50.0%        |
|------|-------------|-------------|
|E     |4.08e+03 mm³ |1.80e+04 mm³ |
|normE |4.08e+03 mm³ |1.80e+04 mm³ |

[ simnibs ]INFO: Finished Running Poslist Number: 1
INFO:simnibs:Finished Running Poslist Number: 1
[ simnibs ]INFO: Result Files:
/home/roman/Документы/Notebooks/Output/simulation_at_pos_59.426521_52.856895_40.114452/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
INFO:simnibs:Result Files:
/home/roman/Документы/Notebooks/Output/simulation_at_pos_59.426521_52.856895_40.1144

starting _0 out of 6
starting _0 out of 6
starting _1 out of 6
starting _1 out of 6
starting _2 out of 6
starting _2 out of 6
starting _3 out of 6
starting _3 out of 6
starting _4 out of 6
starting _4 out of 6
starting _5 out of 6
starting _5 out of 6
Exiting Consumer-67...
Exiting Consumer-68...
Done.
CPU times: user 372 ms, sys: 193 ms, total: 565 ms
Wall time: 15min 13s


In [41]:
%%time
# mulriprocessing 3 cores
result=[]

if __name__ == '__main__':
    #manager = multiprocessing.Manager()
    #results = manager.Queue()
    tasks = multiprocessing.JoinableQueue()
    results = multiprocessing.Queue()

    # spawning consumers with respect to the
    # number cores available in the system -1
    n_consumers = multiprocessing.cpu_count()-1
    print('Spawning %i consumers...' % n_consumers)
    consumers = [Consumer(tasks, results) for i in range(n_consumers)]
    for consumer in consumers:
        consumer.start()

    # enqueueing jobs
    my_input = [(random.uniform(0,100),random.uniform(0,100)) for x in range(10)]
    for item in my_input:
        tasks.put(item)
    # poison pill
    for i in range(n_consumers):
        tasks.put(None)
    
    tasks.join()
    
    result =[results.get() for i in range(len(my_input))]
    print('Done.')

Spawning 3 consumers...
Consumer-64 processing task:
Consumer-65 processing task:
Consumer-66 processing task:


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
[ simnibs ]INFO: Subject Path: None
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
INFO:simnibs:Subject Path: None
INFO:simnibs:Subject Path: None
[ simni

INFO:simnibs:Running PETSc with KSP: cg PC: hypre
[ simnibs ]INFO: Time to prepare the KSP: 2.98s
INFO:simnibs:Time to prepare the KSP: 2.98s
[ simnibs ]INFO: Solving system 1 of 1
INFO:simnibs:Solving system 1 of 1
[ simnibs ]INFO: Running PETSc with KSP: cg PC: hypre
INFO:simnibs:Running PETSc with KSP: cg PC: hypre
[ simnibs ]INFO: Time to prepare the KSP: 3.20s
INFO:simnibs:Time to prepare the KSP: 3.20s
[ simnibs ]INFO: Solving system 1 of 1
INFO:simnibs:Solving system 1 of 1
[ simnibs ]INFO: Running PETSc with KSP: cg PC: hypre
INFO:simnibs:Running PETSc with KSP: cg PC: hypre
[ simnibs ]INFO: Number of iterations: 37 Residual Norm: 8.68e-11
INFO:simnibs:Number of iterations: 37 Residual Norm: 8.68e-11
[ simnibs ]INFO: KSP Converged with reason: 2
INFO:simnibs:KSP Converged with reason: 2
[ simnibs ]INFO: Time to solve system: 26.29s
INFO:simnibs:Time to solve system: 26.29s
[ simnibs ]INFO: Number of iterations: 37 Residual Norm: 8.56e-11
INFO:simnibs:Number of iterations: 37 Re

[ simnibs ]INFO: /home/roman/Документы/Notebooks/Output/simulation_at_pos_38.196838_59.625603_54.552277/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
INFO:simnibs:/home/roman/Документы/Notebooks/Output/simulation_at_pos_46.566605_51.393372_55.326584/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
INFO:simnibs:/home/roman/Документы/Notebooks/Output/simulation_at_pos_37.326508_67.819153_43.353844/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
[ simnibs ]INFO: =====================================
INFO:simnibs:/home/roman/Документы/Notebooks/Output/simulation_at_pos_38.196838_59.625603_54.552277/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
INFO:simnibs:=====================================
[ simnibs ]INFO: =====================================
[ simnibs ]INFO: =====================================
INFO:simnibs:=====================================
INFO:simnibs:=====================================


starting _0 out of 6
starting _1 out of 6
starting _0 out of 6
starting _0 out of 6
starting _2 out of 6
starting _1 out of 6
starting _3 out of 6
starting _1 out of 6
starting _2 out of 6
starting _2 out of 6
starting _4 out of 6
starting _3 out of 6
starting _3 out of 6
starting _5 out of 6
starting _4 out of 6
starting _4 out of 6
starting _5 out of 6
Consumer-65 processing task:


...

Exiting Consumer-65...


[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_53.455193_45.403191_53.438599
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_53.455193_45.403191_53.438599
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began to run TMS simulations
[ simnibs ]INFO: Coil file: /home/roman/anaconda3/envs/simnibs_env/lib/python3.7/site-packages/simnibs/ccd-files/Magstim_70mm_Fig8.nii.gz
INFO:simnibs:Coil file: /home/roman/anaconda3/envs/simnibs_env/lib/python3.7/site-packages/simnibs/ccd-files/Magstim_70mm_Fig8.nii.gz
[ simnibs ]INFO: Using isotropic conductivities
INFO:simnibs:Using isotropic conductivities


Exiting Consumer-66...


[ simnibs ]INFO: Calculating Coil position from (centre, pos_y, distance)
INFO:simnibs:Calculating Coil position from (centre, pos_y, distance)
[ simnibs ]INFO: Matsimnibs: 
[[ -0.2375  -0.6192   0.7485 -50.7337]
 [  0.8817   0.186    0.4336 -67.8709]
 [ -0.4077   0.7629   0.5017 -59.7529]
 [  0.       0.       0.       1.    ]]
INFO:simnibs:Matsimnibs: 
[[ -0.2375  -0.6192   0.7485 -50.7337]
 [  0.8817   0.186    0.4336 -67.8709]
 [ -0.4077   0.7629   0.5017 -59.7529]
 [  0.       0.       0.       1.    ]]
[ simnibs ]INFO: Assembling FEM Matrx
INFO:simnibs:Assembling FEM Matrx
[ simnibs ]INFO: 10.05s to assemble FEM matrix
INFO:simnibs:10.05s to assemble FEM matrix
[ simnibs ]INFO: Calculating dA/dt field
INFO:simnibs:Calculating dA/dt field
DEBUG:simnibs:Solving FEM System
[ simnibs ]INFO: Using solver options: -ksp_type cg -ksp_rtol 1e-10 -pc_type hypre -pc_hypre_type boomeramg -pc_hypre_boomeramg_coarsen_type HMIS
INFO:simnibs:Using solver options: -ksp_type cg -ksp_rtol 1e-10 -pc

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
Exiting Consumer-64...
Done.
CPU times: user 406 ms, sys: 199 ms, total: 606 ms
Wall time: 19min 4s


In [45]:
for x in result:
    print(x)

((88.18419787246357, 20.73120179621355), array([68.045219, 37.519661, 42.394608]), 'hello')
((2.789068869577316, 81.56864381120418), array([40.139816, 76.252769, 27.867176]), 'hello')
((85.89709256243536, 2.793913185425856), array([70.763947, 41.499119, 35.410679]), 'hello')
((72.56203220579968, 16.70041975214285), array([67.101227, 45.456612, 37.561089]), 'hello')
((89.4216826413991, 53.29433385839758), array([48.788033, 38.685085, 62.084385]), 'hello')
((45.467058293190746, 65.85102939695857), array([35.645798, 57.389046, 59.302292]), 'hello')
((31.773505699008155, 22.175446896186447), array([51.413162, 61.83654 , 39.104778]), 'hello')
((17.37969850234252, 35.37628364866059), array([43.610199, 73.678207, 28.99267 ]), 'hello')
((99.38871034032253, 21.77568286797914), array([69.260773, 32.842316, 43.797318]), 'hello')
((58.41724328575197, 19.53173674151335), array([59.426521, 52.856895, 40.114452]), 'hello')


In [48]:
%%time
# mulriprocessing 3 cores (Test 2)
result=[]

if __name__ == '__main__':
    #manager = multiprocessing.Manager()
    #results = manager.Queue()
    tasks = multiprocessing.JoinableQueue()
    results = multiprocessing.Queue()

    # spawning consumers with respect to the
    # number cores available in the system -1
    n_consumers = multiprocessing.cpu_count()-1
    print('Spawning %i consumers...' % n_consumers)
    consumers = [Consumer(tasks, results) for i in range(n_consumers)]
    for consumer in consumers:
        consumer.start()

    # enqueueing jobs
    my_input = [(random.uniform(0,100),random.uniform(0,100)) for x in range(10)]
    for item in my_input:
        tasks.put(item)
    # poison pill
    for i in range(n_consumers):
        tasks.put(None)
    
    tasks.join()
    
    result =[results.get() for i in range(len(my_input))]
    print('Done.')

Spawning 3 consumers...
Consumer-75 processing task:
Consumer-74 processing task:
Consumer-73 processing task:


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Subject Path: None
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_54.768379_69.078339_23.323191
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Simulat

INFO:simnibs:Solving system 1 of 1
[ simnibs ]INFO: Running PETSc with KSP: cg PC: hypre
INFO:simnibs:Running PETSc with KSP: cg PC: hypre
[ simnibs ]INFO: Solving system 1 of 1
INFO:simnibs:Solving system 1 of 1
[ simnibs ]INFO: Running PETSc with KSP: cg PC: hypre
INFO:simnibs:Running PETSc with KSP: cg PC: hypre
[ simnibs ]INFO: Time to prepare the KSP: 3.11s
INFO:simnibs:Time to prepare the KSP: 3.11s
[ simnibs ]INFO: Solving system 1 of 1
INFO:simnibs:Solving system 1 of 1
[ simnibs ]INFO: Running PETSc with KSP: cg PC: hypre
INFO:simnibs:Running PETSc with KSP: cg PC: hypre
[ simnibs ]INFO: Number of iterations: 37 Residual Norm: 8.67e-11
INFO:simnibs:Number of iterations: 37 Residual Norm: 8.67e-11
[ simnibs ]INFO: KSP Converged with reason: 2
INFO:simnibs:KSP Converged with reason: 2
[ simnibs ]INFO: Time to solve system: 25.94s
INFO:simnibs:Time to solve system: 25.94s
[ simnibs ]INFO: Number of iterations: 37 Residual Norm: 8.60e-11
INFO:simnibs:Number of iterations: 37 Resid

INFO:simnibs:Result Files:
/home/roman/Документы/Notebooks/Output/simulation_at_pos_31.82081_61.335064_57.417877/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
[ simnibs ]INFO: =====================================
INFO:simnibs:=====================================
[ simnibs ]INFO: SimNIBS finished running simulations
INFO:simnibs:SimNIBS finished running simulations
[ simnibs ]INFO: Simulation Result Meshes:
INFO:simnibs:Simulation Result Meshes:
[ simnibs ]INFO: /home/roman/Документы/Notebooks/Output/simulation_at_pos_31.82081_61.335064_57.417877/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
INFO:simnibs:/home/roman/Документы/Notebooks/Output/simulation_at_pos_31.82081_61.335064_57.417877/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
[ simnibs ]INFO: =====================================
INFO:simnibs:=====================================


starting _0 out of 6
starting _0 out of 6
starting _0 out of 6
starting _1 out of 6
starting _1 out of 6
starting _1 out of 6
starting _2 out of 6
starting _2 out of 6
starting _2 out of 6
starting _3 out of 6
starting _3 out of 6
starting _3 out of 6
starting _4 out of 6
starting _4 out of 6
starting _4 out of 6
starting _5 out of 6
starting _5 out of 6
starting _5 out of 6
Consumer-73 processing task:


...

Exiting Consumer-75...


[ simnibs ]INFO: 11.72s to assemble FEM matrix
INFO:simnibs:11.72s to assemble FEM matrix
[ simnibs ]INFO: Calculating dA/dt field
INFO:simnibs:Calculating dA/dt field


Exiting Consumer-73...


DEBUG:simnibs:Solving FEM System
[ simnibs ]INFO: Using solver options: -ksp_type cg -ksp_rtol 1e-10 -pc_type hypre -pc_hypre_type boomeramg -pc_hypre_boomeramg_coarsen_type HMIS
INFO:simnibs:Using solver options: -ksp_type cg -ksp_rtol 1e-10 -pc_type hypre -pc_hypre_type boomeramg -pc_hypre_boomeramg_coarsen_type HMIS
[ simnibs ]INFO: Preparing the KSP
INFO:simnibs:Preparing the KSP
[ simnibs ]INFO: Time to prepare the KSP: 4.29s
INFO:simnibs:Time to prepare the KSP: 4.29s
[ simnibs ]INFO: Solving system 1 of 1
INFO:simnibs:Solving system 1 of 1
[ simnibs ]INFO: Running PETSc with KSP: cg PC: hypre
INFO:simnibs:Running PETSc with KSP: cg PC: hypre
[ simnibs ]INFO: Number of iterations: 37 Residual Norm: 8.52e-11
INFO:simnibs:Number of iterations: 37 Residual Norm: 8.52e-11
[ simnibs ]INFO: KSP Converged with reason: 2
INFO:simnibs:KSP Converged with reason: 2
[ simnibs ]INFO: Time to solve system: 11.75s
INFO:simnibs:Time to solve system: 11.75s
[ simnibs ]INFO: Creating visualization

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
Exiting Consumer-74...
Done.
CPU times: user 406 ms, sys: 186 ms, total: 591 ms
Wall time: 18min 59s


---------------------
## Дальше мусор

In [31]:

# mulriprocessing
result=[]

if __name__ == '__main__':
    
    tasks = multiprocessing.JoinableQueue()
    results = multiprocessing.Queue()

    # spawning consumers with respect to the
    # number cores available in the system -1
    n_consumers = multiprocessing.cpu_count()-2
    print('Spawning %i consumers...' % n_consumers)
    consumers = [Consumer(tasks, results) for i in range(n_consumers)]
    for consumer in consumers:
        consumer.start()

    # enqueueing jobs
    my_input = [(random.uniform(0,100),random.uniform(0,100)) for x in range(10)]
    for item in my_input:
        tasks.put(item)
    # poison pill
    for i in range(n_consumers):
        tasks.put(None)
    
    tasks.join()
    
    result =[results.get() for i in range(len(my_input))]
    print('Done.')

Spawning 2 consumers...
Consumer-11 processing task:
Consumer-12 processing task:
Consumer-11 processing task:
Consumer-12 processing task:
Consumer-12 processing task:
Consumer-11 processing task:
Consumer-11 processing task:
Consumer-12 processing task:
Consumer-12 processing task:
Consumer-11 processing task:
Exiting Consumer-12...
Exiting Consumer-11...
Done.


----------------------------

In [52]:

result=[]
for i in range(2):
    # Создаются рандомные углы
    #angles_list = [(random.uniform(0,100),random.uniform(0,100)) for x in range(2)]
    
    # <проверка на существование файла> (перенести ли это в многопроцессорное вычисление?!)
    
    
    # Выполняется многопроцессное вычисление
    if __name__ == '__main__':
        tasks = multiprocessing.JoinableQueue()
        results = multiprocessing.Queue()
        # spawning consumers with respect to the
        # number cores available in the system -1
        n_consumers = multiprocessing.cpu_count()-2
        print('Spawning %i consumers...' % n_consumers)
        consumers = [Consumer(tasks, results) for i in range(n_consumers)]

        for consumer in consumers:
            consumer.start()
        
        print('-1-')
        # enqueueing jobs
        my_input = [(random.uniform(0,100),random.uniform(0,100)) for x in range(2)]
        for item in my_input:
            tasks.put(item)
        # poison pill
        for i in range(n_consumers):
            tasks.put(None)
        print('--2--')
        tasks.join()
        
        print('chek point')
        result = [results.get() for i in range(len(my_input))]
        print('Done.')
        print('well', result)

    # Проискходит сортировка и сужение области поиска для углов
    # <для этого теста не важная часть>

Spawning 2 consumers...
Consumer-34 processing task:
Consumer-33 processing task:


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail


-1-
--2--


[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_54.997204_59.959133_37.334137
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_54.997204_59.959133_37.334137
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_63.620247_52.135933_34.580711
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_63.620247_52.135933_34.580711
[ simnibs ]INFO: Runn

SUMMARY:simnibs:
subject_TMS_1-0001_Magstim_70mm_Fig8_nii
Gray Matter

Field Percentiles
-----------------
Top percentiles of the field (or field norm for vector fields)
|Field |99.9%        |99.0%        |95.0%        |
|------|-------------|-------------|-------------|
|E     |6.53e-01 V/m |4.32e-01 V/m |2.45e-01 V/m |
|normE |6.53e-01 V/m |4.32e-01 V/m |2.45e-01 V/m |

Field Focality
---------------
Mesh volume or area with a field >= X% of the 99.9th percentile
|Field |75.0%        |50.0%        |
|------|-------------|-------------|
|E     |3.15e+03 mm³ |1.33e+04 mm³ |
|normE |3.15e+03 mm³ |1.33e+04 mm³ |

[ simnibs ]INFO: Finished Running Poslist Number: 1
[ simnibs ]INFO: Finished Running Poslist Number: 1
INFO:simnibs:Finished Running Poslist Number: 1
[ simnibs ]INFO: Result Files:
/home/roman/Документы/Notebooks/Output/simulation_at_pos_63.620247_52.135933_34.580711/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
INFO:simnibs:Result Files:
/home/roman/Документы/Notebooks/

starting _0 out of 6
starting _0 out of 6
starting _1 out of 6
starting _1 out of 6
starting _2 out of 6
starting _2 out of 6
starting _3 out of 6
starting _3 out of 6
starting _4 out of 6
starting _4 out of 6
starting _5 out of 6
starting _5 out of 6
Exiting Consumer-34...
Exiting Consumer-33...
chek point
Done.
well [((56.501732881047516, 1.3423719737888362), array([63.620247, 52.135933, 34.580711]), 'hello'), ((41.234848068268924, 17.890188251432505), array([54.997204, 59.959133, 37.334137]), 'hello')]
Spawning 2 consumers...
Consumer-35 processing task:
Consumer-36 processing task:
-1-
--2--


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_46.286663_43.655457_61.391071
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_50.281307_62.79887_38.989178
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_50.2

SUMMARY:simnibs:
subject_TMS_1-0001_Magstim_70mm_Fig8_nii
Gray Matter

Field Percentiles
-----------------
Top percentiles of the field (or field norm for vector fields)
|Field |99.9%        |99.0%        |95.0%        |
|------|-------------|-------------|-------------|
|E     |6.48e-01 V/m |4.49e-01 V/m |2.67e-01 V/m |
|normE |6.48e-01 V/m |4.49e-01 V/m |2.67e-01 V/m |

Field Focality
---------------
Mesh volume or area with a field >= X% of the 99.9th percentile
|Field |75.0%        |50.0%        |
|------|-------------|-------------|
|E     |3.75e+03 mm³ |1.75e+04 mm³ |
|normE |3.75e+03 mm³ |1.75e+04 mm³ |

[ simnibs ]INFO: Finished Running Poslist Number: 1
INFO:simnibs:Finished Running Poslist Number: 1
[ simnibs ]INFO: Result Files:
/home/roman/Документы/Notebooks/Output/simulation_at_pos_46.286663_43.655457_61.391071/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
INFO:simnibs:Result Files:
/home/roman/Документы/Notebooks/Output/simulation_at_pos_46.286663_43.655457_61.3910

starting _0 out of 6
starting _0 out of 6
starting _1 out of 6
starting _1 out of 6
starting _2 out of 6
starting _2 out of 6
starting _3 out of 6
starting _3 out of 6
starting _4 out of 6
starting _4 out of 6
starting _5 out of 6
starting _5 out of 6
Exiting Consumer-36...
Exiting Consumer-35...
chek point
Done.
well [((80.09501335775437, 54.942460633864954), array([46.286663, 43.655457, 61.391071]), 'hello'), ((27.80980768392437, 37.96912744758607), array([50.281307, 62.79887 , 38.989178]), 'hello')]


In [22]:
# Test
calc_data_for_point([79.28632429262, 135.31159610590748])

[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_15.079218_52.303547_70.859001
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_15.079218_52.303547_70.859001


Angles [79.28632429262, 135.31159610590748] is coords: [15.079218 52.303547 70.859001]


[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_15.079218_52.303547_70.859001
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_15.079218_52.303547_70.859001
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began to run TMS simulations
[ simnibs ]INFO: Coil file: /home/roman/anaconda3/envs/simnibs_env/lib/python3.7/site-packages/simnibs/ccd-files/Magstim_70mm_Fig8.nii.gz
INFO:simnibs:Coil file: /home/roman/anaconda3/envs/simnibs_env/lib/python3.7/site-packages/simnibs/ccd-files/Magstim_70mm_Fig8.nii.gz
[ simnibs ]INFO: Using isotropic conductivities
INFO:simnibs:Using isotropic conductivities
[ simnibs ]INFO: Calculating Coil position from (centre, pos_y, distance)
INFO:simnibs:Calculating Coil position from (centre, pos_y, distance)
[ simnibs ]INFO: Matsimnibs: 
[[ -0.1626  

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6


([79.28632429262, 135.31159610590748],
 array([15.0792, 52.3035, 70.859 ]),
 0.3694558257947221)

In [36]:
x,y,z=get_coords_from_angles(55.7457008132232, 117.7437659575659)

In [38]:
out_dir+str(x)+'_'+str(y)+'_'+str(z)+'/'+subject_name+'_effective_field.txt'

'Output/simulation_at_pos_24.190575_57.643375_64.448189/subject_effective_field.txt'

In [39]:
exists(out_dir+str(x)+'_'+str(y)+'_'+str(z)+'/'+subject_name+'_effective_field.txt')

False